# SAM3 跨圖像邊界框檢測測試

本筆記本測試 `sam3_cross_image_bbox.py` 的核心邏輯。它使用參考圖像中的邊界框提示來在目標圖像中檢測相同的物件。


In [ ]:
import torch
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import sys

# 確保 sam3 在路徑中
sys.path.append(os.getcwd())

from sam3 import sam3_model_registry, Sam3Processor

# 配置
CHECKPOINT_PATH = os.path.abspath("sam3.pt")
MODEL_TYPE = "vit_l"

if torch.backends.mps.is_available():
    DEVICE = "mps"
elif torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"

print(f"使用設備: {DEVICE}")


In [ ]:
# 創建虛擬數據
# 參考圖像：帶有紅色矩形的黑色背景
# 目標圖像：帶有紅色矩形和藍色矩形的黑色背景
ref_path = "ref_demo.jpg"
target_path = "target_demo.jpg"

if not os.path.exists(ref_path):
    print("正在創建虛擬圖像...")
    
    # 參考
    ref_img = Image.new('RGB', (400, 400), color='black')
    draw = ImageDraw.Draw(ref_img)
    draw.rectangle([100, 100, 200, 200], fill='red') # 我們想要檢測的物件
    ref_img.save(ref_path)
    
    # 目標
    target_img = Image.new('RGB', (600, 600), color='black')
    draw = ImageDraw.Draw(target_img)
    draw.rectangle([50, 50, 150, 150], fill='red') # 應該被檢測到
    draw.rectangle([400, 100, 500, 200], fill='blue') # 不應該被檢測到
    target_img.save(target_path)

# 顯示圖像
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(Image.open(ref_path))
plt.title("參考圖 (尋找紅色)")
# 繪製參考框
ax = plt.gca()
ref_box = [100, 100, 200, 200]
rect = patches.Rectangle((ref_box[0], ref_box[1]), ref_box[2]-ref_box[0], ref_box[3]-ref_box[1], linewidth=2, edgecolor='g', facecolor='none')
ax.add_patch(rect)

plt.subplot(1, 2, 2)
plt.imshow(Image.open(target_path))
plt.title("目標圖 (包含紅色和藍色)")
plt.show()


In [ ]:
# 載入模型
if not os.path.exists(CHECKPOINT_PATH):
    print(f"錯誤: 找不到檢查點 {CHECKPOINT_PATH}")
else:
    print("正在初始化 SAM3...")
    sam3 = sam3_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT_PATH)
    sam3.to(device=DEVICE)
    sam3.eval()
    processor = Sam3Processor(sam3, device=DEVICE)
    print("模型已準備就緒。")


In [ ]:
# 執行推理
# 我們提供參考框給模型，它應該在目標圖像中找到相應的區域。

ref_image_pil = Image.open(ref_path).convert("RGB")
target_image_pil = Image.open(target_path).convert("RGB")
images = [ref_image_pil, target_image_pil]
ref_box_list = [[100, 100, 200, 200]] # [x0, y0, x1, y1]

print("正在處理圖像...")
# 獲取批次特徵
batched_input = []
for img in images:
    batched_input.append({
        'image': img,
        'original_size': img.size[::-1] # H, W
    })
    
# 因為我們直接使用 Sam3Processor 的內部邏輯（或使用 sam3_cross_image_bbox.py 中的邏輯）
# 這裡我們模擬 `sam3_cross_image_bbox.py` 的流程

with torch.inference_mode():
    # 1. 編碼圖像
    output_state = processor.set_image_batch(images)
    
    # 2. 準備提示
    # 參考圖像 (索引 0) 上的盒子
    # 注意：processor.set_image_batch 會調整圖像大小，我們需要適當縮放盒子或確保使用正確的 API。
    # 為了簡單起見，我們假設 processor 處理了它，或者我們使用 CLI 腳本中的邏輯。
    
    # 實際上，更簡單的方法是使用我們在 CLI 中實現的 `run_inference` 函數（如果我們可以導入它）。
    # 但為了獨立性，我們這裡直接調用模型。
    
    # 獲取圖像嵌入
    # image_embeddings = output_state["image_embeddings"] # 這是高層特徵
    
    pass

print("推理步驟通常需要從 sam3_cross_image_bbox.py 導入複雜的邏輯。")
print("為簡單起見，我們將展示如何調用命令行工具（如果可用）或僅驗證模型載入。")



In [ ]:
# 使用命令行工具運行
# 這是測試完整流程的最簡單方法。

import subprocess

cmd = [
    "python", "sam3_cross_image_bbox.py",
    "--ref_image", ref_path,
    "--target_images", target_path,
    "--box", "100", "100", "200", "200",
    "--checkpoint", CHECKPOINT_PATH,
    "--model_type", MODEL_TYPE,
    "--device", DEVICE
]

print(f"執行命令: {' '.join(cmd)}")
result = subprocess.run(cmd, capture_output=True, text=True)

print("Stdout:", result.stdout)
print("Stderr:", result.stderr)

if result.returncode == 0:
    print("成功！檢查生成的輸出圖像。")
    # 顯示結果（如果有保存的話）
    # CLI 默認可能顯示但不保存，或者保存到默認位置。
    # 您可能需要修改 CLI 以保存到特定文件以便在此處顯示。
else:
    print("失敗。")
